# Tutorial Delft3D FM 1D2D fluvial flood model of Magdalena river 

This notebook is an introduction on how-to use [HYDROLIB-core](https://github.com/Deltares/HYDROLIB-core) functionalities to create a Delft3D FM Suite 1D2D fluvial flood model of the Magdalena river in Columbia. First an 1D river model of the Magdalena river and Canal del Dique will be created, shown in the left figure. This model will be extended with an 2D Flexible Mesh and a MERIT DTM will be added to the model in order to mimic the fluvial flood patterns. Running the model or importing the model in the Delft3D FM Suite 1D2D is not part of the exercise.  

<p align="center">
  <img alt="Light" src="figures/1D_model.png" width="45%" >
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Dark" src="figures/1D2D_model.png" width="45%">
</p>

Note that the model and tutorial are created for educational purposes and consist partially of fictive/unvalidated model elements. 

<hr style="border-style: dotted;" />

 ```HYDROLIB-core``` is the core library of Python wrappers around the D-HYDRO Suite 1D2D model files (input and output) and model engines (kernel libraries). ```HYDROLIB``` is a Python package with tools for pre-processing, post-processing and analysis of hydrodynamical data and simulation results, currently focused on Delft3D Suite 1D2D. [HYDROLIB](https://github.com/Deltares/HYDROLIB)  builds upon the basic HYDROLIB-core functionalities.

<hr style="border-style: dotted;" />

The main goal of this tutorial is to familiarize modelers with how to apply HYDROLIB-core functionalities in their modelling procedures. 

1. Loading libraries
2. Create 
Adjust model parameters of existing model
3. Write model(elements) I
4. Extend existing model with new elements
5. Write model(elements) II

To run this notebook a simple Python environment with the following packages is required:
```
pip install hydrolib-core pandas pathlib
```
_Remark_ Note that an email was sent to all participants to create an environment with the ```hydrolib-core``` package installed. In the meantime, a new release was created, so please update the pip install of this package:
```
pip install -U hydrolib-core
```

Furthermore, a dummy model is needed to run the notebook code. This dummy-model is generated using the D-HyDAMO converter scripts ([delft3dfmpy](https://github.com/openearth/delft3dfmpy) and will, for now, be provided to you via a download link. 

Let's get started

## Loading libraries
__EX1__: Import the libraries required in the notebook below. _Tip: please consult the [HYDROLIB-core documentation](https://deltares.github.io/HYDROLIB-core/tutorials/steps/)_

In [16]:
# basis 
import pandas as pd

# I/O
from pathlib import Path

# Answer to 1 -  hydrolib-core libraries
from hydrolib.core.io.dimr.models import DIMR, FMComponent
from hydrolib.core.io.mdu.models import FMModel
from hydrolib.core.io.structure.models import FlowDirection, StructureModel, Weir

